In [1]:
import pandas as pd

def read_encoded_sas(name):
    for enc in ['utf-8','unicode','iso-8859-1','iso-8859-2','ascii','windows-1250','windows-1252','latin-1']:
        try:
            df=pd.read_sas(name + '.sas7bdat', encoding=enc)
            df.name = name.upper()
        except:
            pass
        else:
            break
    print ("Encoding used: " + enc)
    return df

In [2]:
# Read dataset
dmx=read_encoded_sas('dm')

# Read in PhUSE spreadsheet
deid = pd.read_excel('phuse-deid-standard---sdtm-3.2---v1.01.xlsx', sheet_name='SDTMIG')
deid = deid[['Domain_Prefix','Variable_Name','Direct_Quasi_Identifier (Direct/Quasi)','DI_Primary_Rule','DI_Alternative_Rules','DI_Comment']].fillna('')

Encoding used: iso-8859-1


In [3]:
## Columns into df
cols = pd.DataFrame(dmx.columns.values).reset_index().rename(columns={0:'Variable_Name', 'index':'VarNum'})    # list of columns 
cols['Partial_Name'] = cols['Variable_Name'].apply(lambda var : '--' + var[2:] if var not in ['STUDYID', 'DOMAIN', 'USUBJID', 'POOLID', 'SPDEVID'] else var) # partial names
cols['Domain_Prefix'] = dmx.name
cols.head(n=8)

,VarNum,Variable_Name,Partial_Name,Domain_Prefix
0,0,STUDYID,STUDYID,DM
1,1,DOMAIN,DOMAIN,DM
2,2,USUBJID,USUBJID,DM
3,3,SUBJID,--BJID,DM
4,4,RFSTDTC,--STDTC,DM
5,5,RFENDTC,--ENDTC,DM
6,6,SITEID,--TEID,DM
7,7,INVID,--VID,DM


In [4]:
# First run - direct match of domain and column name
rules1 = cols.merge(right=deid, how='inner')
rules1['order'] = 1

In [5]:
# Second run - match of partial column name (domain = '')
rules2 = (cols
          .merge(right=deid[deid['Domain_Prefix'] == ""], how='inner', left_on='Partial_Name', right_on='Variable_Name', suffixes=['','_y'])
          .drop(['Domain_Prefix_y','Variable_Name_y'], axis=1)
         )
rules2['order'] = 2

In [6]:
# Set rules, select best match
rules = (pd
         .concat([rules1, rules2, cols], sort = False)
         .sort_values(['Domain_Prefix', 'Variable_Name', 'order'])
         .drop_duplicates(subset=['Domain_Prefix', 'Variable_Name'], keep='first')
         .sort_values(['Domain_Prefix', 'VarNum'])         
         .drop('Partial_Name', axis=1)
         .fillna('')
        )
rules

,VarNum,Variable_Name,Domain_Prefix,Direct_Quasi_Identifier (Direct/Quasi),DI_Primary_Rule,DI_Alternative_Rules,DI_Comment,order
0,0,STUDYID,DM,,,,,1.0
1,1,DOMAIN,DM,,,,,1.0
2,2,USUBJID,DM,Direct,Recode subject ID,,,1.0
3,3,SUBJID,DM,Direct,Recode subject ID,,,1.0
4,4,RFSTDTC,DM,Quasi Level 2,Offset,,,1.0
5,5,RFENDTC,DM,Quasi Level 2,Offset,,,1.0
6,6,SITEID,DM,Quasi Level 1,Remove,Recode ID variable,If SITEID is required and is recoded as per th...,1.0
7,7,INVID,DM,Quasi Level 1,Remove,Recode ID variable,If INVID is required and is recoded as per the...,1.0
8,8,INVNAM,DM,Quasi Level 1,Remove,,Such information is related to other individua...,1.0
9,9,BRTHDTC,DM,Quasi Level 1,Remove,,,1.0


In [7]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_multiple.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
dmx.to_excel(writer, sheet_name='DM', index=False)
cols.to_excel(writer, sheet_name='Columns', index=False)
rules.to_excel(writer, sheet_name='Rules', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()